In [7]:
# Initial Imports
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
from datetime import datetime

import requests
import json

load_dotenv()

True

In [2]:
# Retrieving AlphaVantage API Key
alphavantage_api_key = os.getenv("ALPHAVANTAGE_API_KEY")

# Setting AlphaVantage base URL
url_alpha = "https://www.alphavantage.co/query?"

In [69]:
def getIncS(ticker):
    """
    Function for calling AlphaVantage for income statement. Returns a dataframe with the various financials in the quarterly reports
    """
    params = {
        "function":"INCOME_STATEMENT",
        "symbol":ticker,
        "apikey":alphavantage_api_key
    }
    
    response_data = requests.get(url_alpha, params = params).json()
    
    qr_df = pd.DataFrame(response_data["quarterlyReports"])
    qr_df.set_index(qr_df["fiscalDateEnding"], inplace = True)
    qr_df.drop(columns = "fiscalDateEnding", inplace = True)
    
    return qr_df

In [61]:
def getBalS(ticker):
    """
    Function for calling AlphaVantage for balance sheet. Returns a dataframe with the various financials in the quarterly reports
    """
    params = {
        "function":"BALANCE_SHEET",
        "symbol":ticker,
        "apikey":alphavantage_api_key
    }
    
    response_data = requests.get(url_alpha, params = params).json()
    
    qr_df = pd.DataFrame(response_data["quarterlyReports"])
    qr_df.set_index(qr_df["fiscalDateEnding"], inplace = True)
    qr_df.drop(columns = "fiscalDateEnding", inplace = True)
    
    return qr_df

In [62]:
def getSCF(ticker):
    """
    Function for calling AlphaVantage for statement of cash flows. Returns a dataframe with the various financials in the quarterly reports
    """
    params = {
        "function":"BALANCE_SHEET",
        "symbol":ticker,
        "apikey":alphavantage_api_key
    }
    
    response_data = requests.get(url_alpha, params = params).json()
    
    qr_df = pd.DataFrame(response_data["quarterlyReports"])
    qr_df.set_index(qr_df["fiscalDateEnding"], inplace = True)
    qr_df.drop(columns = "fiscalDateEnding", inplace = True)
    
    return qr_df

In [91]:
def getEPS(ticker):
    """
    Function for calling AlphaVantage for earnings (EPS). Returns a dataframe with the various financials in the quarterly reports
    """
    params = {
        "function":"EARNINGS",
        "symbol":ticker,
        "apikey":alphavantage_api_key
    }
    
    response_data = requests.get(url_alpha, params = params).json()
    
    qr_df = pd.DataFrame(response_data["quarterlyEarnings"])
    qr_df.set_index(qr_df["fiscalDateEnding"], inplace = True)
    qr_df.drop(columns = "fiscalDateEnding", inplace = True)
    
    return qr_df

In [89]:
def calcFRatios(ticker):
    """
    Function that calculates the following financial ratios based on the quarterly financial statements of the company:
    - Working capital Ratio
    - Acid-Test Ratio (Quick Ratio)
    - EPS
    - Debt-Equity Ratio
    
    Ratios are returned in a dataframe
    """
    
    # Retrieving Balance Sheet and Earnings
    balS = getBalS(ticker)
    eps = getEPS(ticker)
    
    # Creating return DataFrame
    columns = ["workingCapitalRatio", "acidTestRatio", "eps", "debtEquityRatio", "currentAssets", "currentLiabilities", "inventory", "longTermDebt", "shortTermDebt", "shareholderEquity"]
    ratio_df = pd.DataFrame(index = incS.index, columns = columns)
    
    # Filling out DataFrame with provided data
    ratio_df["currentAssets"] = balS["totalCurrentAssets"].str.replace("None","0").astype("float")
    ratio_df["currentLiabilities"] = balS["totalCurrentLiabilities"].str.replace("None","0").astype("float")
    ratio_df["inventory"] = balS["inventory"].str.replace("None","0").astype("float")
    ratio_df["longTermDebt"] = balS["longTermDebt"].str.replace("None","0").astype("float")
    ratio_df["shortTermDebt"] = balS["shortTermDebt"].str.replace("None","0").astype("float")
    ratio_df["shareholderEquity"] = balS["totalShareholderEquity"].str.replace("None","0").astype("float")
    
    # Calculating the ratios
    
    ratio_df["workingCapitalRatio"] = ratio_df["currentAssets"] / ratio_df["currentLiabilities"]
    ratio_df["acidTestRatio"] = (ratio_df["currentAssets"] - ratio_df["inventory"]) / ratio_df["currentLiabilities"]
    ratio_df["eps"] = eps["reportedEPS"]
    ratio_df["debtEquityRatio"] = (ratio_df["longTermDebt"] + ratio_df["shortTermDebt"]) / ratio_df["shareholderEquity"]
    
    return ratio_df

In [92]:
calcFRatios("AAPL")

,workingCapitalRatio,acidTestRatio,eps,debtEquityRatio,currentAssets,currentLiabilities,inventory,longTermDebt,shortTermDebt,shareholderEquity
fiscalDateEnding,,,,,,,,,,
2020-09-30,1.363604,1.325072,0.73,1.720810,1.437130e+11,1.053920e+11,4.061000e+09,9.866700e+10,1.376900e+10,6.533900e+10
2020-06-30,1.469450,1.427716,0.65,1.559489,1.400650e+11,9.531800e+10,3.978000e+09,9.404800e+10,1.867500e+10,7.228200e+10
2020-03-31,1.495962,1.461267,0.64,1.396328,1.437530e+11,9.609400e+10,3.334000e+09,8.908600e+10,2.042100e+10,7.842500e+10
2019-12-31,1.597782,1.557679,1.25,1.209548,1.632310e+11,1.021610e+11,4.097000e+09,9.307800e+10,1.521400e+10,8.953100e+10
2019-09-30,1.540126,1.501286,0.76,1.194048,1.628190e+11,1.057180e+11,4.106000e+09,9.180700e+10,1.624000e+10,9.048800e+10
2019-06-30,1.504649,1.467248,0.55,1.124015,1.349730e+11,8.970400e+10,3.355000e+09,8.493600e+10,2.348200e+10,9.645600e+10
2019-03-31,1.315382,1.263298,0.62,1.063952,1.233460e+11,9.377200e+10,4.884000e+09,9.020100e+10,2.242900e+10,1.058600e+11
2018-12-31,1.300555,1.254491,1.05,0.973179,1.408280e+11,1.082830e+11,4.988000e+09,9.298900e+10,2.174100e+10,1.178920e+11
2018-09-30,1.123843,1.089992,0.73,1.068467,1.313390e+11,1.168660e+11,3.956000e+09,9.373500e+10,2.074800e+10,1.071470e+11


In [87]:
getEPS("AAPL")

{'symbol': 'AAPL',
 'annualEarnings': [{'fiscalDateEnding': '2020-12-31', 'reportedEPS': '0'},
  {'fiscalDateEnding': '2020-09-30', 'reportedEPS': '3.27'},
  {'fiscalDateEnding': '2019-09-30', 'reportedEPS': '2.98'},
  {'fiscalDateEnding': '2018-09-30', 'reportedEPS': '2.97'},
  {'fiscalDateEnding': '2017-09-30', 'reportedEPS': '2.3'},
  {'fiscalDateEnding': '2016-09-30', 'reportedEPS': '2.0675'},
  {'fiscalDateEnding': '2015-09-30', 'reportedEPS': '2.3'},
  {'fiscalDateEnding': '2014-09-30', 'reportedEPS': '1.6075'},
  {'fiscalDateEnding': '2013-09-30', 'reportedEPS': '1.415'},
  {'fiscalDateEnding': '2012-09-30', 'reportedEPS': '1.5775'},
  {'fiscalDateEnding': '2011-09-30', 'reportedEPS': '0.9875'},
  {'fiscalDateEnding': '2010-09-30', 'reportedEPS': '0.54'},
  {'fiscalDateEnding': '2009-09-30', 'reportedEPS': '0.2225'},
  {'fiscalDateEnding': '2008-09-30', 'reportedEPS': '0.38'},
  {'fiscalDateEnding': '2007-09-30', 'reportedEPS': '0.55'},
  {'fiscalDateEnding': '2006-09-30', 'repo